<a href="https://colab.research.google.com/github/shashank-gajula/robokalam/blob/main/Untitled6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install spacy

In [22]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [27]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load('en_core_web_lg')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [28]:
!curl -L -O http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9684k  100 9684k    0     0  8234k      0  0:00:01  0:00:01 --:--:-- 8234k


In [30]:
!unzip cornell_movie_dialogs_corpus.zip

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-dialogs corpus/movie_characters_metadata.txt  
  inflating: cornell movie-dialogs corpus/movie_conversations.txt  
  inflating: cornell movie-dialogs corpus/movie_lines.txt  
  inflating: cornell movie-dialogs corpus/movie_titles_metadata.txt  
  inflating: cornell movie-dialogs corpus/raw_script_urls.txt  
  inflating: cornell movie-dialogs corpus/README.txt  
  inflating: __MACOSX/cornell movie-dialogs corpus/._README.txt  


In [31]:
movie_lines = {}
for line in open("./cornell movie-dialogs corpus/movie_lines.txt",
                 encoding="latin1"):
    line = line.strip()
    parts = line.split(" +++$+++ ")
    if len(parts) == 5:
        movie_lines[parts[0]] = parts[4]
    else:
        movie_lines[parts[0]] = ""

In [32]:
import json
responses = {}
for line in open("./cornell movie-dialogs corpus/movie_conversations.txt",
                 encoding="latin1"):
    line = line.strip()
    parts = line.split(" +++$+++ ")
    line_ids = json.loads(parts[3].replace("'", '"'))
    for first, second in zip(line_ids[:-1], line_ids[1:]):
        responses[first] = second

In [33]:
import random
for pair in random.sample(responses.items(), 5):
    print("A:", movie_lines[pair[0]])
    print("B:", movie_lines[pair[1]])
    print()

A: Eve. I'd forgotten they grew that way...
B: The lack of pretense, that sort of strange directness and understanding-

A: And if that's not destiny, I don't know what is.
B: That's really romantic.

A: It means I'm gonna shampoo my hair and stay out of your way.
B: Hand me a towel.  Hand me a towel.

A: What if he checks it?
B: He won't -- I mean, he didn't the last time. He'll be expecting fifty thousand and there it is -- on top.

A: We're wasting time.
B: See!  That's what I like about you, man!  You're just sharp as a razor blade.



In [34]:
import numpy as np
def sentence_mean(nlp, s):
    if s == "":
        s = " "
    doc = nlp(s, disable=['tagger', 'parser'])
    return np.mean(np.array([w.vector for w in doc]), axis=0)
sentence_mean(nlp, "This... is a test.").shape

(300,)

In [35]:
!pip install simpleneighbors

In [36]:
from simpleneighbors import SimpleNeighbors

nns = SimpleNeighbors(300)
for i, line_id in enumerate(random.sample(list(responses.keys()), 10000)):
    # show progress
    if i % 1000 == 0: print(i, line_id, movie_lines[line_id])
    line_text = movie_lines[line_id]
    summary_vector = sentence_mean(nlp, line_text)
    if np.any(summary_vector):
        nns.add_one(line_id, summary_vector)
nns.build()

0 L118088 No one thinks you're a piece of shit, Tommy.
1000 L93564 What?
2000 L594344 Get it out of here!
3000 L262794 He's married...
4000 L615790 You're just a Master of Ceremonies. He might have knocked his head off.
5000 L542565 Was there visual contact?
6000 L555465 You went out last night, didn't you, Joe?
7000 L551622 We've prohibited the networks from broadcasting the transmissions.
8000 L258392 I'll come.
9000 L596987 Yes. Friedmans the fuckin Austrian!


In [37]:
sentence = "I like making bots."
picked = nns.nearest(sentence_mean(nlp, sentence), 5)[0]
response_line_id = responses[picked]

print("Your line:\n\t", sentence)
print("Most similar turn:\n\t", movie_lines[picked])
print("Response to most similar turn:\n\t", movie_lines[response_line_id])

Your line:
	 I like making bots.
Most similar turn:
	 I think so.
Response to most similar turn:
	 What do you mean, you think so?


In [38]:
!pip install https://github.com/aparrish/semanticsimilaritychatbot/archive/master.zip

     \ 20kB 23.4MB/s
  Created wheel for semanticsimilaritychatbot: filename=semanticsimilaritychatbot-0.0.1-cp36-none-any.whl size=4844 sha256=26393ab530f20f1b562da95f05488aeb36edd630b39640b16220c538daabffa9
  Stored in directory: /tmp/pip-ephem-wheel-cache-0vtti0gz/wheels/f7/af/8e/8a8fbef31bfbfc3b935425efa03db03825795d85f4e23f8255
Successfully built semanticsimilaritychatbot


In [40]:
from semanticsimilaritychatbot import SemanticSimilarityChatbot
chatbot = SemanticSimilarityChatbot(nlp, 300)

In [41]:
sample_n = 10000
for first_id, second_id in random.sample(list(responses.items()), sample_n):
    chatbot.add_pair(movie_lines[first_id], movie_lines[second_id])
chatbot.build()

In [42]:
print(chatbot.response_for("Hello computer!"))

Grady, it's Sara. Thank God you're there. You won't believe what's happened.


In [43]:
my_turn = "The weather's nice today, don't you think?"
for i in range(5, 51, 5):
    print("picking from", i, "possible responses:")
    print(chatbot.response_for(my_turn, i))
    print()

picking from 5 possible responses:
Lester's left Felicia. He's taking me to Acapulco on the five o'clock flight. He's asked me to marry him.

picking from 10 possible responses:
You are being typically theatrical, Marion. I need the Tribune to--

picking from 15 possible responses:
I guess if I had my choice, I'd like to do something that involves children.  Work with kids in some way.  Teenagers, maybe.  And Santa Carla seems to be full of them.

picking from 20 possible responses:
It's called Tron. It's a security program itself, actually. Monitors all the contacts between our system and other systems... If it finds anything going on that's not scheduled, it shuts it down. I sent you a memo on it.

picking from 25 possible responses:
I'll get there as soon as I can.

picking from 30 possible responses:
No! Not enough!  In times like these, when everything is uncertain, every conservative person should have a substantial part of his fortune within arm's reach.

picking from 35 possibl

In [44]:
chatbot.save("movielines-10k-sample")

In [45]:
chatbot = SemanticSimilarityChatbot.load("movielines-10k-sample", nlp)

In [46]:
print(chatbot.response_for("I'm going to go get some coffee."))

I'm worried about Jake.


In [47]:
print(chatbot.response_for("How are you doing?"))

Hi, Kathleen, I'm just working.


In [48]:
print(chatbot.response_for("i hope you're doing well now"))

What?
